# Captures a video of 10 seconds

In [ ]:
import cv2
import numpy as np
import time

capture_duration = 10
 
# Create a VideoCapture object
cap = cv2.VideoCapture(0)
 
# Check if camera opened successfully
if (cap.isOpened() == False): 
    print("Unable to read camera feed")
 # Default resolutions of the frame are obtained.The default resolutions are system dependent.
# We convert the resolutions from float to integer.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
 
# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
out = cv2.VideoWriter('output.mp4',cv2.VideoWriter_fourcc('M','P','4','V'), 10, (frame_width,frame_height))

start_time = time.time()
while(int(time.time() - start_time) < capture_duration):
    ret, frame = cap.read()
 
    if ret == True: 
     
    # Write the frame into the file 'output.avi'
        out.write(frame)
 
    # Display the resulting frame    
        cv2.imshow('frame',frame)
 
    # Press Q on keyboard to stop recording
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
 
  # Break the loop
    else:
        break 
 # When everything done, release the video capture and video write objects
cap.release()
out.release()
 
# Closes all the frames
cv2.destroyAllWindows() 

# Reads the captured video in frames, prints the total number of blinks 

In [ ]:
import dlib
from scipy.spatial import distance as dist
import imutils
from imutils import face_utils
from imutils.video import FileVideoStream
from imutils.video import VideoStream

def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # compute the euclidean distance between the horizontal eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])

    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    # return the eye aspect ratio
    return ear

In [ ]:
def main() :
    #args = vars(ap.parse_args())
    EYE_AR_THRESH = 0.27
    EYE_AR_CONSEC_FRAMES = 2

    # initialize the frame counters and the total number of blinks
    COUNTER = 0
    TOTAL = 0
    count = 0

    # initialize dlib's face detector (HOG-based) and then create
    # the facial landmark predictor
    print("[INFO] loading facial landmark predictor...")
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
 
    # grab the indexes of the facial landmarks for the left and
    # right eye, respectively
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    
    #cap = cv2.VideoCapture('output.mp4') 
    vs = FileVideoStream('output.mp4').start()
    fileStream = True
    time.sleep(1.0)
    
    while True:
            frame = vs.read()
            if fileStream and not vs.more():
                break
            
            #frame = imutils.resize(frame, width=450)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            #cv2.imshow("Trial", gray)
            
            
            # detect faces in the grayscale frame
            rects = detector(gray, 0)
            
            # loop over the face detections
            for rect in rects:
                # determine the facial landmarks for the face region, then convert the facial landmark (x, y)-coordinates to a NumPy array
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
    
                # extract the left and right eye coordinates, then use the coordinates to compute the eye aspect ratio for both eyes
                leftEye = shape[lStart:lEnd]
                rightEye = shape[rStart:rEnd]
                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)
    
                # average the eye aspect ratio together for both eyes
                ear = (leftEAR + rightEAR) / 2.0
    
                # compute the convex hull for the left and right eye, then visualize each of the eyes
                leftEyeHull = cv2.convexHull(leftEye)
                rightEyeHull = cv2.convexHull(rightEye)
                cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
                
                # check to see if the eye aspect ratio is below the blink threshold, and if so, increment the blink frame counter
                if ear < EYE_AR_THRESH:
                    COUNTER += 1
                # otherwise, the eye aspect ratio is not below the blink threshold
                else:
                    # if the eyes were closed for a sufficient number of then increment the total number of blinks
                    if COUNTER >= EYE_AR_CONSEC_FRAMES:
                        TOTAL += 1
    
                    # reset the eye frame counter
                    COUNTER = 0
                # draw the total number of blinks on the frame along with
                # the computed eye aspect ratio for the frame
                cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            # show the frame
            cv2.imshow("Frame", frame)
            key = cv2.waitKey(1) & 0xFF
     
            # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break
    print(TOTAL)
    
    # do a bit of cleanup
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__' :
    main()